In [1]:
import FinanceDataReader as fdr
import pandas as pd
import pandas_market_calendars as mcal
import warnings
import ta

from pykrx import stock
from defs import filter_df
from PublicDataReader import Fred
from marcap import marcap_data
from functools import reduce

In [2]:
warnings.simplefilter(action='ignore', category=FutureWarning) # FutureWarning 제거

code = '005930'

stock_df = fdr.DataReader(code,'2021-01-01','2023-07-31').reset_index()

buy_df = stock.get_market_trading_volume_by_date("20210101", "20230731", code, on='매수').iloc[:,2].reset_index()
sell_df = stock.get_market_trading_volume_by_date("20210101", "20230731", code, on='매도').iloc[:,2].reset_index()

# 이동평균선 5,20,60,200 O
ma = [5,20,60,120]
for days in ma:
    stock_df['ma_'+str(days)] = stock_df['Close'].rolling(window = days).mean().round(2)

H, L, C, V = stock_df['High'], stock_df['Low'], stock_df['Close'], stock_df['Volume']
#해당 종목 EMA
stock_df['EMA'] = ta.trend.ema_indicator(close=C, fillna=True).round(2)
#해당 종목 SMA
stock_df['SMA'] = ta.trend.sma_indicator(close=C, fillna=True).round(2)
#해당 종목 RSI
stock_df['RSI'] = ta.momentum.rsi(close=C, fillna=True).round(2)
#해당 종목 MFI
stock_df['MFI'] = ta.volume.money_flow_index(high=H, low=L, close=C, volume=V, fillna=True).round(2)
#해당 종목 상승률
stock_df['Change'] = (stock_df['Change']*100).round(2)
#해당 종목 VPT
stock_df['VPT'] = ta.volume.volume_price_trend(close=C, volume=V).round(2)
#해당 종목 일목균형표
stock_df['VI'] = ta.trend.vortex_indicator_pos(high=H,low=L,close=C,fillna=True).round(2)
#y 라벨 결과 데이터프레임 
label_values = (stock_df['Change'] >= 5) & (stock_df['Change'] > 0)
shifted_values = label_values.shift(2)
stock_df['Label'] = shifted_values
#해당 종목 체결강도
stock_df['VP'] = (buy_df['개인']/sell_df['개인']*100).round(2)

#해당 종목 시가총액, 거래대금, 주식수
M_df = marcap_data('2021-01-01','2023-07-31',code=code)
selected_colums = ['Amount','Marcap','Stocks']
M_df = M_df[selected_colums].reset_index()

#데이터 병합
merge_df = [stock_df,M_df]
dataset_df = reduce(lambda x,y : pd.merge(x,y,on='Date'),merge_df)

#주식시장 개장일만 분류
filtered_df = filter_df(dataset_df)